In [1]:
import os
import re
import pickle
import numpy as np
import pandas as pd
from itertools import combinations, product
from operator import itemgetter
from humanize import ordinal
from sklearn.cluster import KMeans
from scipy.spatial.distance import euclidean
from scipy.stats import wilcoxon

In [2]:
pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

In [3]:
RESULTS_PATH = 'data/results'
FEATURES_PATH = 'data/features'

In [4]:
results_dfs = {}
for filename in os.listdir(RESULTS_PATH):
    archtecture = re.search(r'(?<=results_).*(?=.csv)', filename).group()
    results_dfs[archtecture] = pd.read_csv(os.path.join(RESULTS_PATH, filename))

In [5]:
metabase_df = pd.concat(results_dfs.values(), ignore_index=True)

In [6]:
metabase_df.head()

,Architecture,Classes,Accuracy_vector,Precision_vector,Recall_vector,F1_vector,Accuracy_avg,Precision_Avg,Recall_Avg,F1_avg
0,MiniVGG1,airplane-automobile,"[0.97550000000000003, 0.97050000000000003, 0.9...","[0.96317397806580263, 0.95639910089910085, 0.9...","[0.97699999999999998, 0.97099999999999997, 0.9...","[0.97553669495756368, 0.97051474262868564, 0.9...",0.97190,0.956602,0.9766,0.972037
1,MiniVGG1,airplane-bird,"[0.92149999999999999, 0.92000000000000004, 0.9...","[0.88043696275071626, 0.87799999999999989, 0.8...","[0.94499999999999995, 0.94499999999999995, 0.9...","[0.92330239374694667, 0.92195121951219505, 0.9...",0.91925,0.878482,0.9418,0.921148
2,MiniVGG1,airplane-cat,"[0.93899999999999995, 0.95099999999999996, 0.9...","[0.89860873605947966, 0.92032854406130271, 0.8...","[0.97699999999999998, 0.97299999999999998, 0.9...","[0.94123314065510599, 0.95205479452054798, 0.9...",0.94390,0.910412,0.9694,0.945516
3,MiniVGG1,airplane-deer,"[0.85499999999999998, 0.9365, 0.90700000000000...","[0.77534549689441001, 0.89005545536519381, 0.8...","[0.999, 0.99099999999999999, 0.992999999999999...","[0.87325174825174834, 0.9397818871503083, 0.91...",0.89945,0.836805,0.9932,0.908942
4,MiniVGG1,airplane-dog,"[0.95599999999999996, 0.94350000000000001, 0.9...","[0.92603428571428559, 0.90506057781919846, 0.9...","[0.98099999999999998, 0.97999999999999998, 0.9...","[0.95707317073170728, 0.94548962855764584, 0.9...",0.94830,0.914100,0.9797,0.950101


In [7]:
features = {}
for filename in os.listdir(FEATURES_PATH):
    class_ = re.search(r'\w+(?=.data)', filename).group()
    with open(os.path.join(FEATURES_PATH, filename), 'rb') as f:
        features[class_] = pickle.load(f)

In [8]:
centroids = {}
for classes in metabase_df['Classes'].unique():
    class_1, class_2 = classes.split('-')
    X = np.concatenate((features[class_1], features[class_2]), axis=0)
    kmeans = KMeans(n_clusters=2, random_state=0)
    kmeans.fit(X)
    centroids[classes] = kmeans.cluster_centers_

In [9]:
distances = {}
for tasks in combinations(metabase_df['Classes'].unique(), 2):
    task_1, task_2 = tasks
    distances[tasks] = min(
        np.mean([
            euclidean(centroids[task_1][0], centroids[task_2][0]),
            euclidean(centroids[task_1][1], centroids[task_2][1])
        ]),
        np.mean([
            euclidean(centroids[task_1][0], centroids[task_2][1]),
            euclidean(centroids[task_1][1], centroids[task_2][0])
        ])
    )

In [10]:
similar_tasks_by_tasks = {}
for task in metabase_df['Classes'].unique():
    hypotheses = []
    for tasks in distances.keys():
        if task in tasks:
            hypothese = tasks[1] if task == tasks[0] else tasks[0]
            hypotheses.append((hypothese, distances[tasks]))
    similar_tasks_by_tasks[task] = sorted(hypotheses, key=itemgetter(1))

In [11]:
statistics_by_tasks = {}
for task in metabase_df['Classes'].unique():
    df = metabase_df[metabase_df['Classes'] == task].reset_index(drop=True)
    row_with_higher_accuracy = df.iloc[df['Accuracy_avg'].idxmax()].to_dict()
    row_with_lower_accuracy = df.iloc[df['Accuracy_avg'].idxmin()].to_dict()
    ranking_architectures = df.sort_values(
        'Accuracy_avg',
        ascending=False
    )['Architecture'].tolist()
    statistics_by_tasks[task] = {
        'best_architecture': row_with_higher_accuracy,
        'worst_architecture': row_with_lower_accuracy,
        'ranking_architectures': ranking_architectures,
        'amount_of_architectures': len(ranking_architectures),
        'Accuracy_avg_wilcoxon': wilcoxon(df['Accuracy_avg']).pvalue,
        'Precision_Avg_wilcoxon': wilcoxon(df['Precision_Avg']).pvalue,
        'Recall_Avg_wilcoxon': wilcoxon(df['Recall_Avg']).pvalue,
        'F1_avg_wilcoxon': wilcoxon(df['F1_avg']).pvalue
    }

/home/lucas/.cache/pypoetry/virtualenvs/algorithm-recommendation-j2TDJCVt-py3.7/lib/python3.7/site-packages/scipy/stats/morestats.py:2863: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [12]:
for task in metabase_df['Classes'].unique():
    similar_task, centroids_distance = similar_tasks_by_tasks[task][0]
    recommended_architecture = statistics_by_tasks[
        similar_task
    ]['best_architecture']
    statistics_by_tasks[task].update({
        'centroids_distance': centroids_distance,
        'recommended_architecture': recommended_architecture,
        'recommended_architecture_rank': statistics_by_tasks[task][
            'ranking_architectures'
        ].index(recommended_architecture['Architecture'])
    })

In [13]:
recommendations_df = pd.DataFrame(
    metabase_df['Classes'].unique(),
    columns=['Task']
)

In [14]:
recommendations_df['Amount_of_architectures'] = [
    statistics_by_tasks[task]['amount_of_architectures']
    for task in recommendations_df['Task']
]

In [15]:
recommendations_df['Ranking_architectures'] = [
    ', '.join(statistics_by_tasks[task]['ranking_architectures'])
    for task in recommendations_df['Task']
]

In [16]:
recommendations_df['Recommended_architecture_rank'] = [
    ordinal(statistics_by_tasks[task]['recommended_architecture_rank'] + 1)
    for task in recommendations_df['Task']
]

In [17]:
recommendations_df['Centroids_distance'] = [
    statistics_by_tasks[task]['centroids_distance']
    for task in recommendations_df['Task']
]

In [18]:
params = [
    'Architecture',
    'Accuracy_avg',
    'Precision_Avg',
    'Recall_Avg',
    'F1_avg'
]
types = ['recommended', 'best', 'worst']
for p, t in product(params, types):
    recommendations_df[f'{p}[{t}]'] = [
        statistics_by_tasks[task][t + '_architecture'][p]
        for task in recommendations_df['Task']
    ]

In [19]:
for metric in ['Accuracy_avg', 'Precision_Avg', 'Recall_Avg', 'F1_avg']:
    recommendations_df[f'{metric}[wilcoxon]'] = [
        statistics_by_tasks[task][metric + '_wilcoxon']
        for task in recommendations_df['Task']
    ]

In [20]:
recommendations_df

,Task,Amount_of_architectures,Ranking_architectures,Recommended_architecture_rank,Centroids_distance,Architecture[recommended],Architecture[best],Architecture[worst],Accuracy_avg[recommended],Accuracy_avg[best],Accuracy_avg[worst],Precision_Avg[recommended],Precision_Avg[best],Precision_Avg[worst],Recall_Avg[recommended],Recall_Avg[best],Recall_Avg[worst],F1_avg[recommended],F1_avg[best],F1_avg[worst],Accuracy_avg[wilcoxon],Precision_Avg[wilcoxon],Recall_Avg[wilcoxon],F1_avg[wilcoxon]
0,airplane-automobile,6,"MiniVGG1, MiniVGG3, AlexNet1, MiniGoogLeNet1, ...",4th,0.052087,MiniGoogLeNet1,MiniVGG1,SmallSqueezeNet,0.96710,0.97190,0.87925,0.952866,0.956602,0.858420,0.9646,0.9766,0.8080,0.967008,0.972037,0.824166,0.027708,0.027708,0.027708,0.027708
1,airplane-bird,6,"MiniVGG1, MiniGoogLeNet1, AlexNet1, MiniVGG3, ...",2nd,0.037969,MiniGoogLeNet1,MiniVGG1,ResNet10,0.96365,0.91925,0.88480,0.945979,0.878482,0.843588,0.9657,0.9418,0.8753,0.963716,0.921148,0.883662,0.027708,0.027708,0.027708,0.027708
2,airplane-cat,6,"MiniVGG3, AlexNet1, MiniVGG1, MiniGoogLeNet1, ...",1st,0.051728,MiniVGG3,MiniVGG3,SmallSqueezeNet,0.96070,0.95015,0.90460,0.944049,0.920075,0.872486,0.9578,0.9716,0.8927,0.960557,0.951379,0.900057,0.027708,0.027708,0.027708,0.027708
3,airplane-deer,6,"MiniGoogLeNet1, MiniVGG3, AlexNet1, ResNet10, ...",6th,0.037969,MiniVGG1,MiniGoogLeNet1,MiniVGG1,0.91925,0.96365,0.89945,0.878482,0.945979,0.836805,0.9418,0.9657,0.9932,0.921148,0.963716,0.908942,0.027708,0.027708,0.027708,0.027708
4,airplane-dog,6,"MiniGoogLeNet1, MiniVGG3, AlexNet1, MiniVGG1, ...",4th,0.051966,MiniVGG1,MiniGoogLeNet1,SmallSqueezeNet,0.97900,0.96525,0.94280,0.967914,0.953657,0.911598,0.9813,0.9553,0.9620,0.979050,0.964880,0.944186,0.027708,0.027708,0.027708,0.027708
5,airplane-frog,6,"MiniGoogLeNet1, MiniVGG3, AlexNet1, SmallSquee...",1st,0.051883,MiniGoogLeNet1,MiniGoogLeNet1,MiniVGG1,0.98130,0.96560,0.94675,0.974671,0.951720,0.906626,0.9763,0.9610,0.9937,0.981136,0.964927,0.949362,0.027708,0.027708,0.027708,0.027708
6,airplane-horse,6,"MiniVGG3, MiniGoogLeNet1, AlexNet1, MiniVGG1, ...",1st,0.049007,MiniVGG3,MiniVGG3,ResNet10,0.97825,0.96500,0.93480,0.970667,0.942993,0.909471,0.9724,0.9795,0.9278,0.978120,0.965587,0.934305,0.027708,0.027708,0.027708,0.027708
7,airplane-ship,6,"MiniVGG3, MiniGoogLeNet1, AlexNet1, MiniVGG1, ...",4th,0.095961,MiniVGG1,MiniVGG3,SmallSqueezeNet,0.91925,0.93570,0.70285,0.878482,0.904545,0.667966,0.9418,0.9446,0.6881,0.921148,0.936251,0.669603,0.027708,0.027708,0.027708,0.027708
8,airplane-truck,6,"MiniVGG3, MiniGoogLeNet1, AlexNet1, MiniVGG1, ...",1st,0.050565,MiniVGG3,MiniVGG3,SmallSqueezeNet,0.96090,0.94775,0.84080,0.942395,0.915364,0.816722,0.9624,0.9735,0.9400,0.960918,0.949271,0.874130,0.027708,0.027708,0.027708,0.027708
9,automobile-bird,6,"MiniGoogLeNet1, MiniVGG3, AlexNet1, MiniVGG1, ...",3rd,0.042926,AlexNet1,MiniGoogLeNet1,SmallSqueezeNet,0.98565,0.97695,0.90715,0.977944,0.961333,0.890751,0.9874,0.9875,0.8509,0.985685,0.977209,0.856313,0.027708,0.027708,0.027708,0.027708


In [21]:
recommendations_df.to_csv('recommendations.csv', index=False)